# Spring 2018 Status: UNREVIEWED

# What do we mean by loops?

Honestly, this is a good question.  There is a lot of language in programming that was created on the fly and: 

* was never technically correct but understood by the community of practice
* once was technically correct but no longer correct with modern systems, but the term is stil used for historical tradition

So what do we mean when we use the word loop to describe a structure? It can mean many things!  For example, a for loop in Python repeats a block of code over items in a sequence.  

# DEPRECATED While loops

When it comes to data processing in python, the while loop is only deployed when the for loop won't cut it.  

While loops are the standard way to loop for some languages, but not so much in python.  There are certain tasks, such as interactive programs, where the while loop is almost always what you use.

This class is dealing with data, which is usually in files so you know exactly how many times to run.

The while loop is valuable in the cases when you don't know how many times you'll need to run through things.

Even though this isn't something that you'll use a huge amount, you should still know about it.

A for loop goes over all the things and does whatever you want for all of them.  You can read that as "for each in that thing...".  Meanwhile, a while loop gets to loop over and over until it's question is answered correctly.  It has one question and will go until tha question comes up as true.

As soon as that condition is met, the while loop is done.  This is great for conditions of trying to act as a gatekeeper for crappy user input, or waiting for a command from a user.

This is not always a great choice for dealing with data, because we are almost always dealing with a known amount of stuff.

The other limitation of the while loop is that you have to do all the legwork.  You can't just tell it to loop over all the things in a file, because that's the purpose of a for loop.  It will keep repeating the question, and it's up to you to traverse what's going on.

So counters and other incrementers are required.

In [ ]:
import random
count_sum = 0
top = 100
step_count = 0

# generate random integers between 0 and 10, add them until the sum is greater than 100
# report how many times it took to do that
while count_sum < 100:
    my_num =  random.randint(0, 10)

    count_sum += my_num # sum = sum + my_num
    step_count += 1 # step_count = step_count + 1
    print(count_sum, step_count)

We can also have this repeat 100 times to see what the average step count is over 100 runs.

In [ ]:
import random

# priming all the values
count_sum = 0
top = 100
step_count = 0

all_step_counts = []

for i in range(100):
    while count_sum < 100:
        my_num =  random.randint(0, 10)

        count_sum += my_num # sum = sum + my_num
        step_count += 1 # step_count = step_count + 1
#     print(sum, step_count)
    all_step_counts.append(step_count)
    # reset!
    count_sum = 0
    step_count = 0

print(all_step_counts)
print("The average number of steps to take is:", sum(all_step_counts) / len(all_step_counts))

In [ ]:
100 / 5

Do you see that reset process that we had to go through?  We need to reset our counters back to the beginning when we want it to start over.

We'll be exploring a similar application of collecting things repeatedly, but then when that collection is done we want to reset our collectors before it continues along further.

# We aren't going to use for loops for the dracula chapter problem

You could, but it would suck.  Instead, we can take an approach that is inspired by the while loop.

The core problem here is that we need to traverse over the entire document, but the while loop will stop as soon as it finds the thing that it's looking for.  We could use a while loop to find the start of the chapters and the end of the chapters, but we would need a separate while loop for each.  Our problem is that there are 27 things to find, and there are just better ways of doing it.

Let's look at a real data example here.

So we know that there are states happening in the file, but we don't know how many universities will be listed under each.  We can use a combination of pattern here:

* a decision tree to filter the lines we have found into their meaning groups
* a main and sub accumalator pattern that you used in the week where you were going over a file and using an accumulaor on each of the lines, and then collecting all of the results in a final accumulator.

Let's do the easy stuff first.

## pattern 1: filter all the things

Taking a look at the file in question, we can see that there are two kinds of lines:

* a line talking about a state
* a line talking about a university

We need to figure out how to detect these differences.  Going back to our previous discussions:

1. we could try and detect is only a state name is mentioned in a line
2. we could exploit the known and steady sturucture

Let's go for item 2 first.

* some of the universities have () in the names
* some of the universities have footnotes, and therefor end withd [a number]

But is this something regular enough to depend on?

We can test the () first, and let's print out what that might tell us about the states, since that is the smaller category.

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
lines = text.split("\n")
fileio.close()

for line in lines:
    if '(' in line and ')' in line:
        print("----")
    else:
        print("this in a state:", line)

Ouch, we've got some universities in this.  Some of them have the [] in there, so we can add that in next.

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
lines = text.split("\n")
fileio.close()

for line in lines:
    if '(' in line and ')' in line:
        print("----")
    elif '[' in line and ']' in line:
        print("----")
    else:
        print("this in a state:", line)

That made things worse. We've lost the states completely.

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
lines = text.split("\n")
fileio.close()

for line in lines:
    if '(' in line and ')' in line and '[' in line and ']' in line:
        print("----")
    else:
        print("this in a state:", line)

This also didn't work.

Maybe this is time to think about what might be unique about the states. We can eyeball the first few in the file, and we can take a look at the source page this came from to find something.

https://en.wikipedia.org/wiki/List_of_college_towns

Skimming through the wixipedia page shows us that there appearts to be an edit link next to each state name.  Is this duplitated through the file?

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()

print(text.count('[edit]'))
fileio.close()


So there are 51 results!  Yes, there are 50 states but District of Columbia is counted as a state.  Also you can look at the outline of the page and it shows that there are 51 sections.

We might think about now splitting the text on this [edit] text since it does mark the start of a new section. 

However, when we do that we end up with this:

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
lines = text.split("[edit]")

print(lines[:2])
fileio.close()


The state names are now separated from their contents, and the name of the next state is at the end of the previous one's data.

So that sucks.

Since we can't use split here, we might want to just use a filter.  In this case, we know that [edit] comes at the end. So we can use .endswith to check if that's our line.

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
fileio.close()

lines = text.split("\n")

states = []

for line in lines:
    if line.endswith("[edit]"):
        print(line)
        states.append(line)

print(len(states))

There we have it!  We found a filter that correctly identifies our states, without any false positives of false negatives.

Now that we can correctly find the markers for a state, we can turn our attention to what we might want to do about that getting the data under it.

So we have two kinds of line:

1. lines that serve as a marke that a new state is reporting
2. lines of data about college towns, that all belong to the state that above it

We need to decide what we want to do when we hit a nen state line.

## the main/sub loop pattern

Recall our previous pattern that dealt with lines, and you needed to act on those lines with another loop.  For example:

``` python
for line in file:
    loop through stuff in that line
    add that collected data to the broader list
```

Let's consider a final data structure here:


``` python
[ ['state1', 'city1', 'city2'],
  ['state2', 'city3', 'city4'] ]
```

Our primary accumulator is our outermost list (the outside []).  It serves the purpose of collecting all the secondary accumulators.  

## Analogy: making info packets

Let's imagine a situation where you are assembling info packets for visiting candidates.  You have a table lined up with all the papers and packets to include.  You go down that table, grabbing one of each and pacing it on the bottom of your current pile.  At the end of the table, you staple all the papers together.  Finally, you put the packet into the box holding all the completed packets.

That's a pretty defined accumulator pattern.  You just have to keep grabbing all the papers until there are none left to grab.

Our data is a little different.

## Analogy: a sandwich maker

Say you work at a sub sandwich shop.  A customer comes in and puts in an order.  There are a few standard things to do, but then you get to toppings.  Your job is to keep adding toppings to the sandwhich until they tell you they are done. You don't know how many times you'll add stuff to the sandwich, but you'll know when they say they are done.  Then you package up the sandwich and hand it to the customer.

All the customers you have in the day is your primary accumulator, and the secondary accumulators are the single sandwiches that you make.


## OK so this still isn't perfect

If finding a state line tells us that we need to start collecting a new bunch of data, then we need to decide what to do with the stuff we have previously collected.  Like a customer putting a bunch of shoe requests into you, and by the time you get back to them, they are gone and a new customer has a whole different set of requests.  You're still standing there with a bunch of shoes in your hand.  

When you see that new customer, you excuse yourself and put the previous order to the side.  This is more what we are after.  The customer itself does't tell us that we are done, the fact that we see a new customer tells us that the previous order is done.

So here's our plan:

* loop through the lines
* when the line is city (so not a state):
    * add that data to the secondary accumulator
* what the line is a state:
    * add secondary accumulator package to your primary accumulator
    * reset the secondary accumulator back to empty
    * add the state to that newly empty secondary accumulator
    
We've sort of seen this before, where we had:

```python
allstuff = []

for line in file:
    newthing = []
    for something in line:
        ...
        newthing.append(stuff)
    allstuff.append(newthing)
```

`allstuff` is our primary accumulator and `newthing` is our secondary accumulator.  Because there's a very difinite place where we want to start over, we have `newthing = []` just inside our primary for loop, with the secondary loop adding to it.  This allows it to reset everytime we hit a new group.

In our case, we are using a conditional to check when we need to reset things.  That reset process includes adding the currently accrued contents to the primary accumulator, and then setting the secondary accumulator back to empty.  

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
lines = text.split("\n")

allchunks = []
statechunk = []

for line in lines:
    if line.endswith("[edit]"):
        print("a new state has begun!", line)
        # add completed state
        print("beginning the reset process")
        allchunks.append(statechunk)
        # reset the chunk
        statechunk = [line]
    else:
        # otherwise, just keep adding things
        statechunk.append(line)
    allchunks.append(statechunk)
        

In [ ]:
print(allchunks)

Now, this doesn't look all bad.  We've got the things that we want, but we've got an empty list out front.  This is because the first thing that our script hits is a state line, which begins the full reset process.  But that isn't what we want to do.  We havn't collected any data yet, so there's nothing to add to the primary accumulator.  In fact, there's nothing to reset because we haven't done anything!

So we need to add a little boolean check in here.  We'll set it to ta false to start, but then once we've gone through our first state we can set it to True.

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
lines = text.split("\n")

allchunks = []
statechunk = []

foundfirst = False

for line in lines:
    if line.endswith("[edit]") and foundfirst == False:
        print("This is the first state!", line)
        statechunk.append(line)
        foundfirst = True
    elif line.endswith("[edit]"):
        print("a new state has begun!", line)
        # add completed state
        allchunks.append(statechunk)
        # reset the chunk
        statechunk = [line]
    else:
        statechunk.append(line)
    allchunks.append(statechunk)
        

In [ ]:
allchunks[-1]

In [ ]:
fileio = open('crappystatedata.txt', 'r')

text = fileio.read()
lines = text.split("\n")

allchunks = []
statechunk = []

foundfirst = False

for line in lines[:20]:
    print("XXX", line)
    if line.endswith("[edit]"):
        statechunk = []
        statechunk = [line]
        allchunks.append(statechunk)
        print("IF", allchunks)
    else:
        statechunk.append(line)
        print("ELSESTATE", statechunk)
        print("ELSE", allchunks)

In [ ]:
data = """X
a
b
X
c
d
e
"""

lines = data.split()

chunks = []
chunk = []

for line in lines:
    if line == "X":
        print("before", id(chunk))
        chunk = []
        print("after", id(chunk))
        chunks.append(chunk) # I'm bleeding now
        chunk.append(line)
    else:
        print("adding to", id(chunk))
        chunk.append(line)
    print(list(map(id, chunks)))

In [ ]:
chunks

In [ ]:
data = """X
a
b
X
c
d
e
"""

lines = data.split()

chunks = []
chunk = []

for line in lines:
    if line == "X":
        chunk = []
        chunks.append(chunk[:]) # ???
        chunk.append(line)
    else:
        chunk.append(line)
        
print(chunks)